In [1]:
just_checking_integrity=False

In [2]:
import gc
import pandas as pd
import scipy.sparse as sps

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
valid=sps.load_npz("./drive/MyDrive/Recommendation_system/dataset/processed_data/URM_valid_bought.npz")
bought=valid.indices

In [5]:
dataset={}
dataset_max={}
#["UCF","ICBF","ICF","Graph","TopPop100","TopPop50","TopPop15"] # ,"UCF_W","Graph_W","ICF_W","UCF_WW","ICBF_WW","Graph_WW","ICF_WW", "ICBF_W"
for file in ["UCF", "ICBF", "ICF","Graph"]:
    print(gc.collect())
    print(f"loading {file}")
    df=pd.read_csv(f"./drive/MyDrive/Recommendation_system/dataset/candidates/traditional_recs/train/{file}.csv")
    print(len(df))
    print(f"loaded {file}")
    for col in df:
        if "Max" in col:
            df_max=df[["Session_Id",col]].drop_duplicates()
        elif "Score" in col:
            df_feats=df[["Session_Id","Item_ID",col]]
    # if dataset is None:
    dataset[file]=df_feats
    dataset_max[file]=df_max
    # else:
    #     dataset=dataset.merge(df_feats,on=["Session_Id","Item_ID"],how="outer")
    #     dataset_max=dataset_max.merge(df_max,on=["Session_Id"],how="outer")
    del df


#del dataset_max
del df_feats
del df_max

0
loading UCF
18588456
loaded UCF
0
loading ICBF
16931962
loaded ICBF
0
loading ICF
20341641
loaded ICF
0
loading Graph
9381571
loaded Graph


In [6]:
# import ast
# submission_df=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/GRU/train/candidates.csv")
# if just_checking_integrity:
#     submission_df["score"]=submission_df["score"].apply(lambda x:x.replace("-inf","0")).apply(lambda x:x.replace("+inf","0")).apply(lambda x:x.replace("inf","0"))
#     submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# else:
#     submission_df["score"]=submission_df["score"].map(ast.literal_eval)
# submission_df["code"]=submission_df["code"].map(ast.literal_eval)
# submission_df["rank"]=submission_df["rank"].map(ast.literal_eval)
# submission_df = submission_df.explode(['code','score', 'rank'])
# submission_df[["session_id","code","rank"]]=submission_df[["session_id","code","rank"]].astype(int)
# submission_df.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_gru","score":"score_gru"},inplace=True)

In [7]:
# submission_df.head()

In [8]:
# import ast
# submission_df_tr=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/Transformer/train/candidates.csv")
# if just_checking_integrity:
#     submission_df_tr["score"]=submission_df_tr["score"].apply(lambda x:x.replace("-inf","0"))
#     submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# else:
#     submission_df_tr["score"]=submission_df_tr["score"].map(ast.literal_eval)
# submission_df_tr["code"]=submission_df_tr["code"].map(ast.literal_eval)
# submission_df_tr["rank"]=submission_df_tr["rank"].map(ast.literal_eval)
# submission_df_tr = submission_df_tr.explode(['code','score', 'rank'])
# submission_df_tr[["session_id","code","rank"]]=submission_df_tr[["session_id","code","rank"]].astype(int)
# submission_df_tr.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_transformer","score":"score_transformer"},inplace=True)

In [9]:
import ast
submission_df_LSTM=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/candidates/NN/LSTM/train/candidates.csv")
if just_checking_integrity:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].apply(lambda x:x.replace("-inf","0"))
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
else:
    submission_df_LSTM["score"]=submission_df_LSTM["score"].map(ast.literal_eval)
submission_df_LSTM["code"]=submission_df_LSTM["code"].map(ast.literal_eval)
submission_df_LSTM["rank"]=submission_df_LSTM["rank"].map(ast.literal_eval)
submission_df_LSTM = submission_df_LSTM.explode(['code','score', 'rank'])
submission_df_LSTM[["session_id","code","rank"]]=submission_df_LSTM[["session_id","code","rank"]].astype(int)
submission_df_LSTM.rename(columns={"code":"Item_ID","session_id":"Session_Id","rank":"rank_LSTM","score":"score_LSTM"},inplace=True)

In [10]:
for key in dataset:
    # dataset[key]=dataset[key].merge(submission_df,on=["Session_Id","Item_ID"],how="outer")
    # dataset[key]=dataset[key].merge(submission_df_tr,on=["Session_Id","Item_ID"],how="outer")
    dataset[key]=dataset[key].merge(submission_df_LSTM,on=["Session_Id","Item_ID"],how="outer")
    dataset[key]=dataset[key].merge(dataset_max[key],on=["Session_Id"],how="left")


In [11]:
if just_checking_integrity:
    for key in dataset:
        dataset[key]=dataset[key][:int(0.5*len(dataset))]

In [12]:
del dataset_max
# del submission_df
# del submission_df_tr

In [13]:
import numpy as np
sessions_dict = {}
train_dict = {}

for key in dataset:
    sessions=np.unique(dataset[key]["Session_Id"])
    train=pd.DataFrame()
    train["Session_Id"]=sessions
    if just_checking_integrity:
        train["Item_ID"]=bought[:len(sessions)]
    else:
        train["Item_ID"]=bought
    train["target"]=1

    sessions_dict[key] = sessions
    train_dict[key] = train

In [14]:
gc.collect()

484

In [15]:
for key in dataset:
    dataset[key]=dataset[key].merge(train_dict[key],on=["Session_Id","Item_ID"],how="left")
    dataset[key].fillna(0,inplace=True)
gc.collect()

0

In [16]:
for key in dataset:
    dataset[key]["target"]=dataset[key]["target"].astype("int8")
gc.collect()

0

In [17]:
for key in dataset:
    sessions_containing_bought_among_candidates=dataset[key][["Session_Id","target"]].groupby("Session_Id").agg(sum).reset_index()
    sessions_containing_bought_among_candidates=sessions_containing_bought_among_candidates.loc[sessions_containing_bought_among_candidates["target"]>0]

    dataset[key]=dataset[key][dataset[key]['Session_Id'].isin(sessions_containing_bought_among_candidates["Session_Id"].tolist())]

In [18]:
for key in dataset:
    NHITS=len(dataset[key]['Session_Id'].unique())
    HITRATE=NHITS/81600 #approximate value
    print('model:', key)
    print(HITRATE)
    print('----------')

model: UCF
0.6026348039215687
----------
model: ICBF
0.6534436274509804
----------
model: ICF
0.6773406862745098
----------
model: Graph
0.6230882352941176
----------


In [19]:
for key in dataset:
    dataset[key].head(20)

In [20]:
for key in dataset:
    dataset[key]

In [21]:
dataset_macro_features = pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/macro_feats_NN.csv',
                                     usecols=['session_id',
                                                'session_similarity',
                                                'session_similarity_uniques', 'session_similarity_feats', 'session_similarity_feats_uniques',
                                              'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',])

dataset_macro_features.rename(columns={'session_id':'Session_Id'}, inplace=True)

for key in dataset:
    dataset[key] = dataset[key].merge(dataset_macro_features, on='Session_Id', how='left')
del dataset_macro_features

In [22]:
#dataset.drop('Max_Score_TopPop15', axis=1, inplace=True)

## K-Fold for UFC + LSTM models

In [23]:
from sklearn.model_selection import KFold
import lightgbm
import os
import time

start_time = time.time()

k=10

traditional_model_key = 'UCF'

score=0
kf = KFold(n_splits=k)
index=0
for train_index, valid_index in kf.split(train_dict[traditional_model_key]):

    train_sessions=train_dict[traditional_model_key]["Session_Id"][train_index].tolist()
    valid_sessions=train_dict[traditional_model_key]["Session_Id"][valid_index].tolist()

    train_set=dataset[traditional_model_key][dataset[traditional_model_key]["Session_Id"].isin(train_sessions)]
    valid_set=dataset[traditional_model_key][dataset[traditional_model_key]["Session_Id"].isin(valid_sessions)]

    train_set=train_set.sort_values("Session_Id")
    valid_set=valid_set.sort_values("Session_Id")

    qids_train = train_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()
    qids_valid = valid_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()

    X_train=train_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_train=train_set["target"]

    X_valid=valid_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_valid=valid_set["target"]

    model = lightgbm.LGBMRanker(
    objective="lambdarank",
    #objective="rank_xendcg",
    #metric="mrr",
    n_estimators=2000,
    lambdarank_truncation_level=103,
    reg_alpha=1e-4,
    learning_rate=0.05,
    min_child_samples=400,
    subsample=0.8,
    subsample_freq=20
    )
    ranker=model.fit(
        X=X_train,
        y=y_train,
        group=qids_train,
        eval_set=[(X_valid, y_valid)],
        eval_group=[qids_valid],
        eval_at=(100),
        callbacks=[lightgbm.log_evaluation(),lightgbm.early_stopping(20,first_metric_only=True)],
        #verbose=10,
        eval_metric=["map","ndcg"]
    )
    ranker.booster_.save_model(f'./drive/MyDrive/Recommendation_system/rankers/model_{index}.txt')
    #Get maximum val score
    score+=np.max(ranker._evals_result["valid_0"]["map@100"])
    index+=1

end_time = time.time()

elapsed_time = end_time - start_time

[LightGBM] [Info] Total groups: 43783, total data: 12362623
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.447382 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2801
[LightGBM] [Info] Number of data points in the train set: 12362623, number of used features: 13
[LightGBM] [Info] Total groups: 5392, total data: 1528273
[1]	valid_0's map@100: 0.222161	valid_0's ndcg@100: 0.348635
Training until validation scores don't improve for 20 rounds
[2]	valid_0's map@100: 0.257394	valid_0's ndcg@100: 0.376594
[3]	valid_0's map@100: 0.25991	valid_0's ndcg@100: 0.37893
[4]	valid_0's map@100: 0.26135	valid_0's ndcg@100: 0.380049
[5]	valid_0's map@100: 0.260938	valid_0's ndcg@100: 0.379857
[6]	valid_0's map@100: 0.261254	valid_0's ndcg@100: 0.380112
[7]	valid_0's map@100: 0.261307	valid_0's ndcg@100: 0.380326
[8]	valid_0's map@100: 0.259867	valid_0's

In [24]:
final_performcance=score/k
print("overall MAP@100 =", final_performcance)

overall MAP@100 = 0.28780717106222614


In [25]:
print("normalized score =", final_performcance*HITRATE)

normalized score = 0.1793292623221547


In [26]:
print("K-Fold execution time:", elapsed_time, "seconds")

K-Fold execution time: 765.5759007930756 seconds


## K-Fold for ICBF + LSTM models

In [27]:
start_time = time.time()

k=10

traditional_model_key = 'ICBF'

score=0
kf = KFold(n_splits=k)
index=0
for train_index, valid_index in kf.split(train_dict[traditional_model_key]):

    train_sessions=train_dict[traditional_model_key]["Session_Id"][train_index].tolist()
    valid_sessions=train_dict[traditional_model_key]["Session_Id"][valid_index].tolist()

    train_set=dataset[traditional_model_key][dataset[traditional_model_key]["Session_Id"].isin(train_sessions)]
    valid_set=dataset[traditional_model_key][dataset[traditional_model_key]["Session_Id"].isin(valid_sessions)]

    train_set=train_set.sort_values("Session_Id")
    valid_set=valid_set.sort_values("Session_Id")

    qids_train = train_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()
    qids_valid = valid_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()

    X_train=train_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_train=train_set["target"]

    X_valid=valid_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_valid=valid_set["target"]

    model = lightgbm.LGBMRanker(
    objective="lambdarank",
    #objective="rank_xendcg",
    #metric="mrr",
    n_estimators=2000,
    lambdarank_truncation_level=103,
    reg_alpha=1e-4,
    learning_rate=0.05,
    min_child_samples=400,
    subsample=0.8,
    subsample_freq=20
    )
    ranker=model.fit(
        X=X_train,
        y=y_train,
        group=qids_train,
        eval_set=[(X_valid, y_valid)],
        eval_group=[qids_valid],
        eval_at=(100),
        callbacks=[lightgbm.log_evaluation(),lightgbm.early_stopping(20,first_metric_only=True)],
        #verbose=10,
        eval_metric=["map","ndcg"]
    )
    ranker.booster_.save_model(f'./drive/MyDrive/Recommendation_system/rankers/model_{index}.txt')
    #Get maximum val score
    score+=np.max(ranker._evals_result["valid_0"]["map@100"])
    index+=1

end_time = time.time()

elapsed_time = end_time - start_time

[LightGBM] [Info] Total groups: 47983, total data: 13133374
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.495954 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2799
[LightGBM] [Info] Number of data points in the train set: 13133374, number of used features: 13
[LightGBM] [Info] Total groups: 5338, total data: 1476478
[1]	valid_0's map@100: 0.214295	valid_0's ndcg@100: 0.334823
Training until validation scores don't improve for 20 rounds
[2]	valid_0's map@100: 0.242568	valid_0's ndcg@100: 0.358451
[3]	valid_0's map@100: 0.24374	valid_0's ndcg@100: 0.359188
[4]	valid_0's map@100: 0.244779	valid_0's ndcg@100: 0.360171
[5]	valid_0's map@100: 0.246713	valid_0's ndcg@100: 0.361525
[6]	valid_0's map@100: 0.247112	valid_0's ndcg@100: 0.361588
[7]	valid_0's map@100: 0.24704	valid_0's ndcg@100: 0.361458
[8]	valid_0's map@100: 0.248397	valid_0'

In [28]:
final_performcance=score/k
print("overall MAP@100 = ", final_performcance)

overall MAP@100 =  0.25663969352269833


In [29]:
print("normalized score ", final_performcance*HITRATE)

normalized score  0.1599091737434813


In [30]:
print("K-Fold execution time:", elapsed_time, "seconds")

K-Fold execution time: 1225.5413825511932 seconds


## K-Fold for ICF + LSTM models

In [31]:
start_time = time.time()

k=10

traditional_model_key = 'ICBF'

score=0
kf = KFold(n_splits=k)
index=0
for train_index, valid_index in kf.split(train_dict[traditional_model_key]):

    train_sessions=train_dict[traditional_model_key]["Session_Id"][train_index].tolist()
    valid_sessions=train_dict[traditional_model_key]["Session_Id"][valid_index].tolist()

    train_set=dataset[traditional_model_key][dataset[traditional_model_key]["Session_Id"].isin(train_sessions)]
    valid_set=dataset[traditional_model_key][dataset[traditional_model_key]["Session_Id"].isin(valid_sessions)]

    train_set=train_set.sort_values("Session_Id")
    valid_set=valid_set.sort_values("Session_Id")

    qids_train = train_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()
    qids_valid = valid_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()

    X_train=train_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_train=train_set["target"]

    X_valid=valid_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_valid=valid_set["target"]

    model = lightgbm.LGBMRanker(
    objective="lambdarank",
    #objective="rank_xendcg",
    #metric="mrr",
    n_estimators=2000,
    lambdarank_truncation_level=103,
    reg_alpha=1e-4,
    learning_rate=0.05,
    min_child_samples=400,
    subsample=0.8,
    subsample_freq=20
    )
    ranker=model.fit(
        X=X_train,
        y=y_train,
        group=qids_train,
        eval_set=[(X_valid, y_valid)],
        eval_group=[qids_valid],
        eval_at=(100),
        callbacks=[lightgbm.log_evaluation(),lightgbm.early_stopping(20,first_metric_only=True)],
        #verbose=10,
        eval_metric=["map","ndcg"]
    )
    ranker.booster_.save_model(f'./drive/MyDrive/Recommendation_system/rankers/model_{index}.txt')
    #Get maximum val score
    score+=np.max(ranker._evals_result["valid_0"]["map@100"])
    index+=1

end_time = time.time()

elapsed_time = end_time - start_time

[LightGBM] [Info] Total groups: 47983, total data: 13133374
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.458559 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2799
[LightGBM] [Info] Number of data points in the train set: 13133374, number of used features: 13
[LightGBM] [Info] Total groups: 5338, total data: 1476478
[1]	valid_0's map@100: 0.214295	valid_0's ndcg@100: 0.334823
Training until validation scores don't improve for 20 rounds
[2]	valid_0's map@100: 0.242568	valid_0's ndcg@100: 0.358451
[3]	valid_0's map@100: 0.24374	valid_0's ndcg@100: 0.359188
[4]	valid_0's map@100: 0.244779	valid_0's ndcg@100: 0.360171
[5]	valid_0's map@100: 0.246713	valid_0's ndcg@100: 0.361525
[6]	valid_0's map@100: 0.247112	valid_0's ndcg@100: 0.361588
[7]	valid_0's map@100: 0.24704	valid_0's ndcg@100: 0.361458
[8]	valid_0's map@100: 0.248397	valid_0'

In [32]:
final_performcance=score/k
print("overall MAP@100 = ", final_performcance)

overall MAP@100 =  0.2566396935226983


In [33]:
print("normalized score ", final_performcance*HITRATE)

normalized score  0.15990917374348126


In [34]:
print("K-Fold execution time:", elapsed_time, "seconds")

K-Fold execution time: 1221.244992017746 seconds


## K-Fold for Graph + LSTM models

In [35]:
start_time = time.time()

k=10

traditional_model_key = 'Graph'

score=0
kf = KFold(n_splits=k)
index=0
for train_index, valid_index in kf.split(train_dict[traditional_model_key]):

    train_sessions=train_dict[traditional_model_key]["Session_Id"][train_index].tolist()
    valid_sessions=train_dict[traditional_model_key]["Session_Id"][valid_index].tolist()

    train_set=dataset[traditional_model_key][dataset[traditional_model_key]["Session_Id"].isin(train_sessions)]
    valid_set=dataset[traditional_model_key][dataset[traditional_model_key]["Session_Id"].isin(valid_sessions)]

    train_set=train_set.sort_values("Session_Id")
    valid_set=valid_set.sort_values("Session_Id")

    qids_train = train_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()
    qids_valid = valid_set[["Session_Id"]].groupby("Session_Id")["Session_Id"].count().to_numpy()

    X_train=train_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_train=train_set["target"]

    X_valid=valid_set.drop(["target","Session_Id","Item_ID"],axis=1)
    y_valid=valid_set["target"]

    model = lightgbm.LGBMRanker(
    objective="lambdarank",
    #objective="rank_xendcg",
    #metric="mrr",
    n_estimators=2000,
    lambdarank_truncation_level=103,
    reg_alpha=1e-4,
    learning_rate=0.05,
    min_child_samples=400,
    subsample=0.8,
    subsample_freq=20
    )
    ranker=model.fit(
        X=X_train,
        y=y_train,
        group=qids_train,
        eval_set=[(X_valid, y_valid)],
        eval_group=[qids_valid],
        eval_at=(100),
        callbacks=[lightgbm.log_evaluation(),lightgbm.early_stopping(20,first_metric_only=True)],
        #verbose=10,
        eval_metric=["map","ndcg"]
    )
    ranker.booster_.save_model(f'./drive/MyDrive/Recommendation_system/rankers/model_{index}.txt')
    #Get maximum val score
    score+=np.max(ranker._evals_result["valid_0"]["map@100"])
    index+=1

end_time = time.time()

elapsed_time = end_time - start_time

[LightGBM] [Info] Total groups: 45543, total data: 7893849
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.121041 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2797
[LightGBM] [Info] Number of data points in the train set: 7893849, number of used features: 13
[LightGBM] [Info] Total groups: 5301, total data: 889539
[1]	valid_0's map@100: 0.236252	valid_0's ndcg@100: 0.362849
Training until validation scores don't improve for 20 rounds
[2]	valid_0's map@100: 0.263927	valid_0's ndcg@100: 0.385504
[3]	valid_0's map@100: 0.26514	valid_0's ndcg@100: 0.386798
[4]	valid_0's map@100: 0.266577	valid_0's ndcg@100: 0.387926
[5]	valid_0's map@100: 0.266215	valid_0's ndcg@100: 0.387663
[6]	valid_0's map@100: 0.266337	valid_0's ndcg@100: 0.387786
[7]	valid_0's map@100: 0.266496	valid_0's ndcg@100: 0.387909
[8]	valid_0's map@100: 0.266454	valid_0's 

In [36]:
final_performcance=score/k
print("overall MAP@100 = ", final_performcance)

overall MAP@100 =  0.28816237551867174


In [37]:
print("normalized score ", final_performcance*HITRATE)

normalized score  0.17955058604009003


In [38]:
print("K-Fold execution time:", elapsed_time, "seconds")

K-Fold execution time: 642.8529417514801 seconds
